# AgentWorkflow & FunctionAgent Beginner Guide

This notebook walks you through setting up and using a basic `AgentWorkflow` with a single `FunctionAgent` using the `llama-index` framework.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/FunctionAgent_Workflow_Basics.ipynb)

# Dependencies

In [ ]:
%pip install llama-index tavily-python

## Setup OpenAI LLM

In [12]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", api_key="Enter your api key here")  # Replace with your OpenAI API key


## Define Web Search Tool

In [13]:
from tavily import AsyncTavilyClient

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key="Enter your api key here")  # Replace with your Tavily API key
    return str(await client.search(query))


## Create FunctionAgent

In [14]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)


## Run the Agent

In [15]:
response = await agent.run(user_msg="What is the weather in San Francisco?")
print(str(response))


The current weather in San Francisco is as follows:

- **Temperature**: 13.3°C (55.9°F)
- **Condition**: Mist
- **Wind**: 8.3 mph (13.3 kph) from the WSW
- **Humidity**: 90%
- **Visibility**: 16 km (9 miles)
- **Feels Like**: 12.2°C (53.9°F)

For more details, you can check the [Weather API](https://www.weatherapi.com/).


## Use AgentWorkflow

In [16]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow(agents=[agent])
response = await workflow.run(user_msg="What is the weather in San Francisco?")
print(str(response))


The current weather in San Francisco is as follows:

- **Temperature**: 13.3°C (55.9°F)
- **Condition**: Mist
- **Wind**: 8.3 mph (13.3 kph) from the WSW
- **Humidity**: 90%
- **Visibility**: 16 km (9 miles)
- **Feels Like**: 12.2°C (53.9°F)

For more details, you can check the [Weather API](https://www.weatherapi.com/).


## Maintain Context State

In [17]:
from llama_index.core.workflow import Context

ctx = Context(agent)
response = await agent.run(user_msg="My name is Logan, nice to meet you!", ctx=ctx)
print(str(response))

response = await agent.run(user_msg="What is my name?", ctx=ctx)
print(str(response))


Nice to meet you, Logan! How can I assist you today?
Your name is Logan.


## Serialize Context

In [18]:
from llama_index.core.workflow import JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())
restored_ctx = Context.from_dict(agent, ctx_dict, serializer=JsonSerializer())

response = await agent.run(user_msg="Do you still remember my name?", ctx=restored_ctx)
print(str(response))


Yes, I remember your name is Logan.
